In [1]:
## ref: https://pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/

In [2]:
import numpy as np
import json
from utils import (
    load_itk, world_to_pixel, any_match,
    calc_iou, nms_pytorch
)
from tqdm import tqdm
import pandas as pd
import torch

In [3]:
data_list_file_path = '/data/output/validation/test_parsed.json'
f = open(data_list_file_path)
data = json.load(f)

print(data['test'][0].keys())

dict_keys(['true_box', 'pred_box', 'score', 'image'])


In [16]:
metrics = []

# for content in tqdm(data['test']):
for index, content in enumerate(data['test'][4:5]):

    img, origin, spacing = load_itk(content['image'])
    origin = np.array(list(reversed(origin)))
    spacing = np.array(list(reversed(spacing)))

    P = []
    targets = []
    
    for true_box in content['true_box']:
        world_coord_cent = np.array((true_box[0], true_box[1], true_box[2]))
        world_coord_diam = np.array((true_box[3], true_box[4], true_box[5]))

        voxel_min, voxel_max = world_to_pixel(
            origin, spacing, world_coord_cent, world_coord_diam
        )

        targets.append({
            "voxel_min": voxel_min.astype(int),
            "voxel_max": voxel_max.astype(int)
        })

    for pred_box, score in zip(content['pred_box'], content['score']):

        # if score > 0.2:
        world_coord_cent = np.array((pred_box[0], pred_box[1], pred_box[2]))
        world_coord_diam = np.array((pred_box[3], pred_box[4], pred_box[5]))

        voxel_min, voxel_max = world_to_pixel(
            origin, spacing, world_coord_cent, world_coord_diam
        )

        P.append([
            voxel_min[0].astype(int), 
            voxel_min[1].astype(int), 
            voxel_min[2].astype(int),
            voxel_max[0].astype(int), 
            voxel_max[1].astype(int), 
            voxel_max[2].astype(int),
            score
        ])

    ### calculate NMS
    filtered_boxes = nms_pytorch(torch.tensor(P), 1e-45)
    print(filtered_boxes)

    if len(targets) > 1:
        print(index, targets)

    ### calculate IoU for each exam
    # area_overlap = 0
    # area_union = 0
    # for prediction in predictions:
    #     for target in targets:
    #         if any_match(prediction, target):
    #             metrics.append({
    #                 "image": content['image'],
    #                 "IoU": calc_iou(target, prediction)
    #             })

[tensor([289.0000, 181.0000,  95.0000, 310.0000, 202.0000, 108.0000,   0.9873]), tensor([305.0000, 306.0000, 239.0000, 321.0000, 323.0000, 248.0000,   0.9238]), tensor([113.0000, 149.0000,  69.0000, 131.0000, 168.0000,  80.0000,   0.8281]), tensor([267.0000, 395.0000, 104.0000, 296.0000, 424.0000, 122.0000,   0.7998]), tensor([ 53.0000, 313.0000,  74.0000,  69.0000, 330.0000,  84.0000,   0.6953]), tensor([373.0000, 164.0000, 165.0000, 385.0000, 176.0000, 171.0000,   0.5596]), tensor([ 98.0000, 324.0000, 109.0000, 112.0000, 338.0000, 117.0000,   0.5503]), tensor([163.0000, 162.0000, 153.0000, 252.0000, 253.0000, 193.0000,   0.4102]), tensor([167.0000, 282.0000, 197.0000, 190.0000, 305.0000, 210.0000,   0.3281]), tensor([2.8200e+02, 4.3000e+02, 4.0000e+01, 3.0700e+02, 4.5400e+02, 5.5000e+01,
        3.2275e-01]), tensor([1.1100e+02, 2.5600e+02, 1.6400e+02, 1.2600e+02, 2.7100e+02, 1.7400e+02,
        2.4011e-01]), tensor([3.7200e+02, 2.6300e+02, 1.0900e+02, 3.8200e+02, 2.7400e+02, 1.1600e